In [1]:
import pandas as pd
import numpy as np
import os
import glob
import os
import subprocess
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
from nipype.interfaces.fsl import ImageMaths
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [2]:
project_dir = '/Users/xiaoqianxiao/projects/OCD'
data_dir = os.path.join(project_dir, 'results/NW_group_network_network')
result_dir = os.path.join(project_dir, 'results/visualization/NW_group_network_network')
if not os.path.exists(result_dir):
    os.makedirs(result_dir, exist_ok=True)

In [10]:
group_diff_file = os.path.join(data_dir, 'group_diff_baseline_schaefer_2018_100_7_2_network_fc.csv')
df_group_diff = pd.read_csv(group_diff_file)
df_group_diff['network1'] = df_group_diff['Feature'].str.split('_').str[0]
df_group_diff['network2'] = df_group_diff['Feature'].str.split('_').str[1]
df_group_diff.sort_values(by='p_value', ascending=True).head(20)

,Feature,t_statistic,p_value,OCD_mean,HC_mean,OCD_n,HC_n,p_value_fdr,network1,network2
22,Default_Limbic_fc,1.589576,0.118627,0.288945,0.231864,46,46,0.740840,Default,Limbic
0,Default_Visual_fc,1.432909,0.158610,0.288282,0.241198,46,46,0.740840,Default,Visual
18,Default_Ventral Attention_fc,1.394988,0.169775,0.347307,0.306912,46,46,0.740840,Default,Ventral Attention
23,Frontoparietal_Limbic_fc,1.343599,0.186390,0.368808,0.324595,46,46,0.740840,Frontoparietal,Limbic
13,Default_Dorsal Attention_fc,1.321679,0.192072,0.350505,0.316847,46,46,0.740840,Default,Dorsal Attention
25,Default_Frontoparietal_fc,1.267367,0.211068,0.355827,0.316710,46,46,0.740840,Default,Frontoparietal
27,Default_Default_fc,1.230624,0.223390,0.464773,0.435554,46,46,0.740840,Default,Default
4,Somatomotor_Visual_fc,1.142396,0.259243,0.353516,0.320220,46,46,0.740840,Somatomotor,Visual
7,Default_Somatomotor_fc,1.109851,0.272348,0.285755,0.247730,46,46,0.740840,Default,Somatomotor
2,Frontoparietal_Visual_fc,1.076780,0.287946,0.309891,0.276181,46,46,0.740840,Frontoparietal,Visual


In [11]:
df_group_diff.loc[
    (df_group_diff['p_value'] <= 0.05)
]

,Feature,t_statistic,p_value,OCD_mean,HC_mean,OCD_n,HC_n,p_value_fdr,network1,network2


In [12]:
df_group_diff.loc[
    (df_group_diff['network1'].isin(['Default', 'Dorsal Attention'])) &
    (df_group_diff['network2'].isin(['Default', 'Dorsal Attention'])) &
    (df_group_diff['p_value'] <= 0.05)]

,Feature,t_statistic,p_value,OCD_mean,HC_mean,OCD_n,HC_n,p_value_fdr,network1,network2


In [13]:
baselineFC_vs_deltaYBOCS_file = os.path.join(data_dir, 'baselineFC_vs_deltaYBOCS_schaefer_2018_100_7_2_network_fc.csv')
df_baselineFC_vs_deltaYBOCS = pd.read_csv(baselineFC_vs_deltaYBOCS_file)
df_baselineFC_vs_deltaYBOCS['network1'] = df_baselineFC_vs_deltaYBOCS['Feature'].str.split('_').str[0]
df_baselineFC_vs_deltaYBOCS['network2'] = df_baselineFC_vs_deltaYBOCS['Feature'].str.split('_').str[1]
df_baselineFC_vs_deltaYBOCS.sort_values(by='fc_p_value', ascending=True).head(20)

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2
21,Ventral Attention_Ventral Attention_fc,0.002792,0.264305,0.172479,44,{'Wait': -0.007995147401680038},{'Wait': 0.7777446154740033},0.031655,-0.015581,0.979196,Ventral Attention,Ventral Attention
26,Frontoparietal_Frontoparietal_fc,-0.002585,0.375912,-0.142734,44,{'Wait': -0.04199178454419143},{'Wait': 0.2095658489175331},0.057721,0.011756,0.979196,Frontoparietal,Frontoparietal
5,Ventral Attention_Visual_fc,0.002220,0.417360,0.120529,44,{'Wait': -0.029226664567265918},{'Wait': 0.34975744664377606},0.035572,-0.011473,0.979196,Ventral Attention,Visual
18,Default_Ventral Attention_fc,0.002311,0.439788,0.116944,44,{'Wait': -0.021575911695356068},{'Wait': 0.5266049084786362},0.023391,-0.024248,0.979196,Default,Ventral Attention
11,Somatomotor_Somatomotor_fc,-0.002064,0.467822,-0.118441,44,{'Wait': -0.04016626625400771},{'Wait': 0.21821282202487047},0.050251,0.003922,0.979196,Somatomotor,Somatomotor
6,Visual_Visual_fc,0.001852,0.470876,0.100965,44,{'Wait': -0.04693326490835969},{'Wait': 0.11348252621612195},0.069553,0.024166,0.979196,Visual,Visual
8,Dorsal Attention_Somatomotor_fc,-0.002288,0.482520,-0.114769,44,{'Wait': -0.0452641135625206},{'Wait': 0.2260697316562269},0.048236,0.001809,0.979196,Dorsal Attention,Somatomotor
9,Frontoparietal_Somatomotor_fc,-0.001866,0.527124,-0.104221,44,{'Wait': -0.042882662795311874},{'Wait': 0.2059621620907504},0.049163,0.002781,0.979196,Frontoparietal,Somatomotor
25,Default_Frontoparietal_fc,-0.001839,0.575388,-0.093049,44,{'Wait': -0.046059200104920935},{'Wait': 0.22207556233944564},0.044484,-0.002126,0.979196,Default,Frontoparietal
15,Dorsal Attention_Frontoparietal_fc,-0.001818,0.578328,-0.092021,44,{'Wait': -0.04119770386293506},{'Wait': 0.27217058837520475},0.037550,-0.009399,0.979196,Dorsal Attention,Frontoparietal


In [14]:
df_baselineFC_vs_deltaYBOCS.loc[
    (df_baselineFC_vs_deltaYBOCS['network1'].isin(['Default', 'Dorsal Attention'])) &
    (df_baselineFC_vs_deltaYBOCS['network2'].isin(['Default', 'Dorsal Attention'])) &
    (df_baselineFC_vs_deltaYBOCS['fc_p_value'] <= 0.05)
]

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2


In [15]:
deltaFC_vs_deltaYBOCS_file = os.path.join(data_dir, 'deltaFC_vs_deltaYBOCS_schaefer_2018_100_7_2_network_fc.csv')
df_deltaFC_vs_deltaYBOCS = pd.read_csv(deltaFC_vs_deltaYBOCS_file)
df_deltaFC_vs_deltaYBOCS['network1'] = df_deltaFC_vs_deltaYBOCS['Feature'].str.split('_').str[0]
df_deltaFC_vs_deltaYBOCS['network2'] = df_deltaFC_vs_deltaYBOCS['Feature'].str.split('_').str[1]
df_deltaFC_vs_deltaYBOCS.sort_values(by='fc_p_value', ascending=True).head(20)

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2
25,Ventral Attention_Ventral Attention_fc,-0.003648,0.182411,-0.211275,43,{'Wait': -0.02031801943731809},{'Wait': 0.5153571626573183},0.054817,0.007558,0.972678,Ventral Attention,Ventral Attention
13,Frontoparietal_Frontoparietal_fc,0.003194,0.219591,0.195085,43,{'Wait': 0.02890993112795364},{'Wait': 0.33271481948216874},0.060637,0.013669,0.972678,Frontoparietal,Frontoparietal
11,Dorsal Attention_Ventral Attention_fc,-0.002750,0.284682,-0.170495,43,{'Wait': -0.014612686586961977},{'Wait': 0.619567330949647},0.035108,-0.013137,0.972678,Dorsal Attention,Ventral Attention
1,Default_Dorsal Attention_fc,-0.002495,0.336872,-0.147522,43,{'Wait': 0.024137080082188594},{'Wait': 0.41884900375180967},0.037813,-0.010296,0.972678,Default,Dorsal Attention
14,Frontoparietal_Limbic_fc,0.002690,0.359285,0.146845,43,{'Wait': 0.019619395834300852},{'Wait': 0.5601786436139053},0.029934,-0.018570,0.972678,Frontoparietal,Limbic
5,Default_Ventral Attention_fc,-0.002548,0.383293,-0.137883,43,{'Wait': 0.001727274001182946},{'Wait': 0.9588640147609426},0.019078,-0.029968,0.972678,Default,Ventral Attention
2,Default_Frontoparietal_fc,0.002387,0.408082,0.133422,43,{'Wait': 0.031193131502744335},{'Wait': 0.3484771336868352},0.039415,-0.008615,0.972678,Default,Frontoparietal
22,Somatomotor_Somatomotor_fc,0.001900,0.461549,0.118615,43,{'Wait': 0.016932062428569185},{'Wait': 0.5682461708445978},0.022163,-0.026729,0.972678,Somatomotor,Somatomotor
17,Frontoparietal_Visual_fc,0.002012,0.497155,0.108936,43,{'Wait': 0.01119374137958092},{'Wait': 0.7422833579044539},0.014568,-0.034703,0.972678,Frontoparietal,Visual
15,Frontoparietal_Somatomotor_fc,0.001787,0.501598,0.108790,43,{'Wait': 0.020944151674497168},{'Wait': 0.49403991431956984},0.023466,-0.025361,0.972678,Frontoparietal,Somatomotor


In [16]:
df_deltaFC_vs_deltaYBOCS.loc[
    (df_deltaFC_vs_deltaYBOCS['network1'].isin(['Default', 'Dorsal Attention'])) &
    (df_deltaFC_vs_deltaYBOCS['network2'].isin(['Default', 'Dorsal Attention'])) &
    (df_deltaFC_vs_deltaYBOCS['fc_p_value'] <= 0.05)
]

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2


In [18]:
condition_baseline_FC_file = os.path.join(data_dir, 'condition_baseline_FC_schaefer_2018_100_7_2_network_fc.csv')
condition_baseline_FC = pd.read_csv(condition_baseline_FC_file)
condition_baseline_FC['network1'] = condition_baseline_FC['Feature'].str.split('_').str[0]
condition_baseline_FC['network2'] = condition_baseline_FC['Feature'].str.split('_').str[1]
condition_baseline_FC.sort_values(by='p_value', ascending=True).head(20)

,Feature,condition1,condition2,t_statistic,p_value,condition1_mean,condition2_mean,condition1_n,condition2_n,effect_size,p_value_fdr,network1,network2
0,Default_Default_fc,Treat,Wait,-1.638766,0.108394,0.489215,0.438109,24,22,-0.481639,0.546878,Default,Default
2,Default_Frontoparietal_fc,Treat,Wait,-1.436931,0.157944,0.380179,0.329262,24,22,-0.419916,0.546878,Default,Frontoparietal
1,Default_Dorsal Attention_fc,Treat,Wait,-1.417504,0.163381,0.371446,0.327660,24,22,-0.416408,0.546878,Default,Dorsal Attention
4,Default_Somatomotor_fc,Treat,Wait,-1.351081,0.183652,0.311930,0.257200,24,22,-0.395388,0.546878,Default,Somatomotor
13,Frontoparietal_Frontoparietal_fc,Treat,Wait,-1.324619,0.192137,0.414282,0.371860,24,22,-0.389366,0.546878,Frontoparietal,Frontoparietal
27,Visual_Visual_fc,Treat,Wait,-1.280017,0.207388,0.476916,0.440446,24,22,-0.373934,0.546878,Visual,Visual
15,Frontoparietal_Somatomotor_fc,Treat,Wait,-1.234422,0.223599,0.391028,0.351236,24,22,-0.363150,0.546878,Frontoparietal,Somatomotor
10,Dorsal Attention_Somatomotor_fc,Treat,Wait,-1.214303,0.231244,0.303115,0.260899,24,22,-0.354743,0.546878,Dorsal Attention,Somatomotor
8,Dorsal Attention_Frontoparietal_fc,Treat,Wait,-1.189020,0.240885,0.288520,0.247159,24,22,-0.347892,0.546878,Dorsal Attention,Frontoparietal
19,Limbic_Somatomotor_fc,Treat,Wait,-1.177593,0.245289,0.401086,0.360024,24,22,-0.346218,0.546878,Limbic,Somatomotor


In [19]:
condition_followup_FC_file = os.path.join(data_dir, 'condition_followup_FC_schaefer_2018_100_7_2_network_fc.csv')
condition_followup_FC = pd.read_csv(condition_followup_FC_file)
condition_followup_FC['network1'] = condition_followup_FC['Feature'].str.split('_').str[0]
condition_followup_FC['network2'] = condition_followup_FC['Feature'].str.split('_').str[1]
condition_followup_FC.sort_values(by='p_value', ascending=True).head(20)

,Feature,condition1,condition2,t_statistic,p_value,condition1_mean,condition2_mean,condition1_n,condition2_n,effect_size,p_value_fdr,network1,network2
27,Visual_Visual_fc,Treat,Wait,-1.441864,0.156942,0.484117,0.440681,21,22,-0.439080,0.707945,Visual,Visual
21,Limbic_Visual_fc,Treat,Wait,-1.338089,0.188286,0.451332,0.406200,21,22,-0.407006,0.707945,Limbic,Visual
19,Limbic_Somatomotor_fc,Treat,Wait,-1.318653,0.194642,0.407313,0.357852,21,22,-0.401147,0.707945,Limbic,Somatomotor
24,Somatomotor_Visual_fc,Treat,Wait,-1.281435,0.207293,0.373674,0.322787,21,22,-0.389706,0.707945,Somatomotor,Visual
25,Ventral Attention_Ventral Attention_fc,Treat,Wait,-1.045379,0.302309,0.404857,0.372214,21,22,-0.320359,0.707945,Ventral Attention,Ventral Attention
11,Dorsal Attention_Ventral Attention_fc,Treat,Wait,-1.035095,0.306928,0.361256,0.325669,21,22,-0.316925,0.707945,Dorsal Attention,Ventral Attention
18,Limbic_Limbic_fc,Treat,Wait,-0.979064,0.333343,0.471884,0.437423,21,22,-0.297679,0.707945,Limbic,Limbic
23,Somatomotor_Ventral Attention_fc,Treat,Wait,-0.856448,0.396728,0.290403,0.254406,21,22,-0.261020,0.707945,Somatomotor,Ventral Attention
12,Dorsal Attention_Visual_fc,Treat,Wait,-0.856241,0.397053,0.286302,0.252676,21,22,-0.262218,0.707945,Dorsal Attention,Visual
20,Limbic_Ventral Attention_fc,Treat,Wait,-0.823042,0.415373,0.312930,0.281634,21,22,-0.251790,0.707945,Limbic,Ventral Attention


In [20]:
condition_FC_change_file = os.path.join(data_dir, 'condition_FC_change_schaefer_2018_100_7_2_network_fc.csv')
condition_FC_change = pd.read_csv(condition_FC_change_file)
condition_FC_change['network1'] = condition_FC_change['Feature'].str.split('_').str[0]
condition_FC_change['network2'] = condition_FC_change['Feature'].str.split('_').str[1]
condition_FC_change.sort_values(by='p_value', ascending=True).head(20)

,Feature,condition1,condition2,t_statistic,p_value,condition1_mean,condition2_mean,condition1_n,condition2_n,effect_size,p_value_fdr,network1,network2
0,Default_Default_fc,Treat,Wait,1.054063,0.298172,-0.040245,-0.004686,21,22,0.320038,0.996939,Default,Default
13,Frontoparietal_Frontoparietal_fc,Treat,Wait,1.005155,0.320778,-0.034171,-0.004307,21,22,0.307044,0.996939,Frontoparietal,Frontoparietal
2,Default_Frontoparietal_fc,Treat,Wait,0.975411,0.335077,-0.037324,-0.005417,21,22,0.297234,0.996939,Default,Frontoparietal
1,Default_Dorsal Attention_fc,Treat,Wait,0.795975,0.430721,-0.030234,-0.006842,21,22,0.241757,0.996939,Default,Dorsal Attention
10,Dorsal Attention_Somatomotor_fc,Treat,Wait,0.721327,0.474804,-0.025747,-0.002323,21,22,0.219949,0.996939,Dorsal Attention,Somatomotor
4,Default_Somatomotor_fc,Treat,Wait,0.712725,0.480129,-0.043498,-0.017940,21,22,0.217907,0.996939,Default,Somatomotor
15,Frontoparietal_Somatomotor_fc,Treat,Wait,0.711246,0.481034,-0.027812,-0.006334,21,22,0.217450,0.996939,Frontoparietal,Somatomotor
25,Ventral Attention_Ventral Attention_fc,Treat,Wait,-0.684248,0.497694,-0.001152,-0.022560,21,22,-0.208917,0.996939,Ventral Attention,Ventral Attention
8,Dorsal Attention_Frontoparietal_fc,Treat,Wait,0.682729,0.498626,-0.030191,-0.008673,21,22,0.208282,0.996939,Dorsal Attention,Frontoparietal
3,Default_Limbic_fc,Treat,Wait,0.663936,0.510546,-0.032199,-0.005948,21,22,0.203111,0.996939,Default,Limbic
